<a href="https://colab.research.google.com/github/moonjune/test-repo/blob/master/knlp_estimator_prac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NLP-kr/tensorflow-ml-nlp.git

Cloning into 'tensorflow-ml-nlp'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 842 (delta 30), reused 27 (delta 15), pack-reused 778
Receiving objects: 100% (842/842), 160.14 MiB | 26.08 MiB/s, done.
Resolving deltas: 100% (497/497), done.
Checking out files: 100% (100/100), done.


In [2]:
import os
os.chdir('/content/tensorflow-ml-nlp')
!pip install -r requirements.txt

     |████████████████████████████████| 19.4MB 4.8MB/s 
     |████████████████████████████████| 174kB 45.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/0e/2b/e8/c0b818ac4b3d35104d35e48cdc7afe27fc06ea277feed2831a
Successfully built JPype1


In [0]:
def model_fn(features, labels, mode, params, config):
  
  # 모델 구현 부분
  estimator = tf.estimator.Estimator(model_fn = model_fn, model_dir = ...,
                                     configs = ..., params = ...)
  
  return tf.estimator.EstimatrSpec( ... )


In [5]:
"""
모델을 학습하는 명령어
estimator.train(input_fn = ...)

학습한 모델을 검증
estimator.evaluate(input_fn = ...)

학습한 모델을 통한 예측
estimator.predict(input_fn = ...)
"""


'\n모델을 학습하는 명령어\nestimator.train(input_fn = ...)\n\n학습한 모델을 검증\nestimator.evaluate(input_fn = ...)\n\n학습한 모델을 통한 예측\nestimator.predict(input_fn = ...)\n'

In [0]:
def train_input_fn():
  
  # 데이터 파이프라인 구현 부분
  
  return features, labels

# tf.data에서 사용한 셔플 배치, 반복 등의 기능 이용 예정
# 학습, 검증, 예측 등 상황에 따라 필요 기능이 다르므로 각 기능에 따라 따로 구현

In [0]:
#패키지와 데이터
import tensorflow as tf
from tensorflow.keras import preprocessing

samples = ['너 오늘 이뻐 보인다', 
           '나는 오늘 기분이 더러워', 
           '끝내주는데, 좋은 일이 있나봐', 
           '나 좋은 일이 생겼어', 
           '아 오늘 진짜 짜증나', 
           '환상적인데, 정말 좋은거 같아']

labels = [[1], [0], [1], [1], [0], [1]]

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index

In [0]:
# 데이터 파이프라인
EPOCH = 100

def train_input_fn():
  
  dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
  dataset = dataset.repeat(EPOCH)
  dataset = dataset.batch(1)
  dataset = dataset.shuffle(len(sequences))
  iterator = dataset.make_one_shot_iterator()
  
  return iterator.get_next()  

# 이 함수가 뱉는 것은 실행하면 다음 데이터 행을 뱉는 객체이다.

In [0]:
# 에스티메이터

VOCAB_SIZE = len(word_index) + 1  # word_index는 딕셔너리로 띄어쓰기로 구분된 모든 단어들(중복제외)의 집합이며, 그 길이가 dlrjt
EMB_SIZE = 128 # EMB_SIZE는 에.. 128개로 고정... 이따 보자

def model_fn(features, labels, mode):
  
  TRAIN = mode == tf.estimator.ModeKeys.TRAIN # loss, train_op필요
  EVAL = mode == tf.estimator.ModeKeys.EVAL # loss 필요
  PREDICT = mode == tf.estimator.ModeKeys.PREDICT # prediction 필요
  # 각 모드에 따라 
  
  embed_input = tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE)(features) 
  embed_input = tf.reduce_mean(embed_input, axis = 1)
  # embedding은 텍스트의 수치화, 존재하는 텍스트 길이와 128개 요인으로 만드는 듯
  # embedding layer는 128개에서 발생한 수치를 평균화하여 원래 VOCAB_SIZE로 만든다.
  # 가 아니네;;; vocab_size를 평균화해버리고 요인 128개 차원의 데이터로 만들어 줌;
  # 얘는 아직 기본 DNN이고 RNN이 아니다 모든 단어를 128개 요인의 수치로 표현된 다음 그걸 요인별로 평균화 함
  # 아직 word2vec 학습이 되거나 한건 아니다. 추후 얘의 정체 알 필요
  
  hidden_layer = tf.keras.layers.Dense(128, activation = tf.nn.relu)(embed_input)
  output_layer = tf.keras.layers.Dense(1)(hidden_layer)
  output = tf.nn.sigmoid(output_layer)
  # 128개의 노드로 받아서 함. 렐루로 값을 보냄
  # dense_layer_2는 정의된 적이 없음.. 함수 밖에서 가져올 수 있나?
  
  loss = tf.losses.mean_squared_error(output, labels)
  # 지도학습이므로 차이를 mean squared error로 계산
  
  if TRAIN:
    global_step = tf.train.get_global_step()
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss, global_step)
    # global_step은 backprop을 하기 위한 것으로 보임
    # train.op는 옵티마이저이고 loss값을 global_step으로 미분???
    # global step은 옵션값으로 미니마이즈 될 변수가 1회 업데이트 된 후 1 증가하는 것으로 step 함수
    
    return tf.estimator.EstimatorSpec(mode = mode,
                                     train_op = train_op,
                                     loss = loss)
  
  # estimator의 스펙을 결정, train이니 train_op와 loss가 필요

In [22]:
# 실제 학습
DATA_OUT_PATH = './data_out/'

import os

if not os.path.exists(DATA_OUT_PATH):
  os.makedirs(DATA_OUT_PATH)
  
estimator = tf.estimator.Estimator(model_fn = model_fn, model_dir = DATA_OUT_PATH + 'checkpoint/dnn')
estimator.train(train_input_fn)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_out/checkpoint/dnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4c138c07b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
IN